In [ ]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

[Learn the Basics](intro.html) \|\|
[Quickstart](quickstart_tutorial.html) \|\|
[Tensors](tensorqs_tutorial.html) \|\| [Datasets &
DataLoaders](data_tutorial.html) \|\|
[Transforms](transforms_tutorial.html) \|\| **Build Model** \|\|
[Autograd](autogradqs_tutorial.html) \|\|
[Optimization](optimization_tutorial.html) \|\| [Save & Load
Model](saveloadrun_tutorial.html)

Build the Neural Network
========================

Neural networks comprise of layers/modules that perform operations on
data. The [torch.nn](https://pytorch.org/docs/stable/nn.html) namespace
provides all the building blocks you need to build your own neural
network. Every module in PyTorch subclasses the
[nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html).
A neural network is a module itself that consists of other modules
(layers). This nested structure allows for building and managing complex
architectures easily.

In the following sections, we\'ll build a neural network to classify
images in the FashionMNIST dataset.


In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

Get Device for Training
=======================

We want to be able to train our model on an
[accelerator](https://pytorch.org/docs/stable/torch.html#accelerators)
such as CUDA, MPS, MTIA, or XPU. If the current accelerator is
available, we will use it. Otherwise, we use the CPU.


In [2]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


Define the Class
================

We define our neural network by subclassing `nn.Module`, and initialize
the neural network layers in `__init__`. Every `nn.Module` subclass
implements the operations on input data in the `forward` method.


In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

We create an instance of `NeuralNetwork`, and move it to the `device`,
and print its structure.


In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To use the model, we pass it the input data. This executes the model\'s
`forward`, along with some [background
operations](https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866).
Do not call `model.forward()` directly!

Calling the model on the input returns a 2-dimensional tensor with dim=0
corresponding to each output of 10 raw predicted values for each class,
and dim=1 corresponding to the individual values of each output. We get
the prediction probabilities by passing it through an instance of the
`nn.Softmax` module.


In [5]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([6])


------------------------------------------------------------------------


Model Layers
============

Let\'s break down the layers in the FashionMNIST model. To illustrate
it, we will take a sample minibatch of 3 images of size 28x28 and see
what happens to it as we pass it through the network.


In [6]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [17]:
print(input_image[0,:,:])

tensor([[0.1135, 0.3164, 0.8281, 0.4287, 0.5358, 0.2895, 0.1469, 0.1261, 0.9152,
         0.4554, 0.5903, 0.6484, 0.3690, 0.6503, 0.7353, 0.6353, 0.2531, 0.1727,
         0.0422, 0.6580, 0.8446, 0.9510, 0.8699, 0.4595, 0.3337, 0.5957, 0.4962,
         0.6378],
        [0.8947, 0.2226, 0.5206, 0.3192, 0.1897, 0.0930, 0.3995, 0.4421, 0.4678,
         0.1840, 0.9274, 0.1405, 0.9438, 0.3619, 0.1571, 0.1173, 0.9463, 0.9152,
         0.8144, 0.3443, 0.1758, 0.3623, 0.8519, 0.7817, 0.7083, 0.9608, 0.7351,
         0.8836],
        [0.8643, 0.7474, 0.8914, 0.7392, 0.6260, 0.1055, 0.8378, 0.8720, 0.1033,
         0.0695, 0.8465, 0.1824, 0.2974, 0.6578, 0.8705, 0.8257, 0.0951, 0.5630,
         0.7993, 0.0257, 0.6355, 0.2804, 0.2704, 0.9936, 0.3917, 0.4255, 0.8408,
         0.2419],
        [0.7235, 0.9931, 0.8618, 0.0099, 0.3404, 0.8504, 0.4706, 0.5057, 0.8969,
         0.4246, 0.5233, 0.4289, 0.7911, 0.9999, 0.6711, 0.0439, 0.3083, 0.4495,
         0.4261, 0.0715, 0.6632, 0.0472, 0.7078, 0.8399

nn.Flatten
==========

We initialize the
[nn.Flatten](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html)
layer to convert each 2D 28x28 image into a contiguous array of 784
pixel values ( the minibatch dimension (at dim=0) is maintained).


In [18]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())
print(flat_image[0,:])

torch.Size([3, 784])
tensor([0.1135, 0.3164, 0.8281, 0.4287, 0.5358, 0.2895, 0.1469, 0.1261, 0.9152,
        0.4554, 0.5903, 0.6484, 0.3690, 0.6503, 0.7353, 0.6353, 0.2531, 0.1727,
        0.0422, 0.6580, 0.8446, 0.9510, 0.8699, 0.4595, 0.3337, 0.5957, 0.4962,
        0.6378, 0.8947, 0.2226, 0.5206, 0.3192, 0.1897, 0.0930, 0.3995, 0.4421,
        0.4678, 0.1840, 0.9274, 0.1405, 0.9438, 0.3619, 0.1571, 0.1173, 0.9463,
        0.9152, 0.8144, 0.3443, 0.1758, 0.3623, 0.8519, 0.7817, 0.7083, 0.9608,
        0.7351, 0.8836, 0.8643, 0.7474, 0.8914, 0.7392, 0.6260, 0.1055, 0.8378,
        0.8720, 0.1033, 0.0695, 0.8465, 0.1824, 0.2974, 0.6578, 0.8705, 0.8257,
        0.0951, 0.5630, 0.7993, 0.0257, 0.6355, 0.2804, 0.2704, 0.9936, 0.3917,
        0.4255, 0.8408, 0.2419, 0.7235, 0.9931, 0.8618, 0.0099, 0.3404, 0.8504,
        0.4706, 0.5057, 0.8969, 0.4246, 0.5233, 0.4289, 0.7911, 0.9999, 0.6711,
        0.0439, 0.3083, 0.4495, 0.4261, 0.0715, 0.6632, 0.0472, 0.7078, 0.8399,
        0.5325, 0.9

nn.Linear
=========

The [linear
layer](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)
is a module that applies a linear transformation on the input using its
stored weights and biases.


In [8]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


nn.ReLU
=======

Non-linear activations are what create the complex mappings between the
model\'s inputs and outputs. They are applied after linear
transformations to introduce *nonlinearity*, helping neural networks
learn a wide variety of phenomena.

In this model, we use
[nn.ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html)
between our linear layers, but there\'s other activations to introduce
non-linearity in your model.


In [9]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.1994, -0.1603, -0.4447,  0.3568,  0.0298,  0.3376,  0.2353, -0.0549,
         -0.2487, -0.0442, -0.0521,  0.1456,  0.1278, -0.0381,  0.2933, -0.3332,
         -0.5531, -0.4041, -0.0293,  0.0159],
        [-0.6827,  0.0952, -0.5183, -0.0834, -0.2009,  0.4886,  0.1158,  0.1032,
          0.0063,  0.2800, -0.0853, -0.1937,  0.2947, -0.1860,  0.2841, -0.3013,
         -0.8171, -0.4126, -0.3823, -0.0673],
        [-0.1918,  0.2631, -0.2424, -0.0861, -0.2826,  0.0991,  0.3610, -0.1972,
         -0.3754, -0.0395, -0.2175, -0.0921,  0.6085,  0.1993, -0.0888, -0.6357,
         -0.4215, -0.4035, -0.1804,  0.2023]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.0000, 0.3568, 0.0298, 0.3376, 0.2353, 0.0000, 0.0000,
         0.0000, 0.0000, 0.1456, 0.1278, 0.0000, 0.2933, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0159],
        [0.0000, 0.0952, 0.0000, 0.0000, 0.0000, 0.4886, 0.1158, 0.1032, 0.0063,
         0.2800, 0.0000, 0.0000, 0.2947, 0.0000, 0.28

nn.Sequential
=============

[nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html)
is an ordered container of modules. The data is passed through all the
modules in the same order as defined. You can use sequential containers
to put together a quick network like `seq_modules`.


In [10]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [11]:
print(input_image)
print(logits)

tensor([[[0.1135, 0.3164, 0.8281,  ..., 0.5957, 0.4962, 0.6378],
         [0.8947, 0.2226, 0.5206,  ..., 0.9608, 0.7351, 0.8836],
         [0.8643, 0.7474, 0.8914,  ..., 0.4255, 0.8408, 0.2419],
         ...,
         [0.3210, 0.2226, 0.2095,  ..., 0.4032, 0.4006, 0.7062],
         [0.8027, 0.9252, 0.6259,  ..., 0.8292, 0.6305, 0.7683],
         [0.8884, 0.7473, 0.8838,  ..., 0.0377, 0.3645, 0.6664]],

        [[0.9962, 0.8877, 0.0047,  ..., 0.6993, 0.5356, 0.3198],
         [0.7102, 0.3253, 0.4189,  ..., 0.5765, 0.5519, 0.8552],
         [0.6896, 0.8164, 0.6124,  ..., 0.8906, 0.7304, 0.6199],
         ...,
         [0.2450, 0.1443, 0.6660,  ..., 0.9829, 0.1814, 0.5767],
         [0.0673, 0.9138, 0.3999,  ..., 0.1373, 0.9186, 0.8238],
         [0.0387, 0.5599, 0.2214,  ..., 0.8037, 0.3250, 0.2255]],

        [[0.5718, 0.0831, 0.3239,  ..., 0.1859, 0.0696, 0.5957],
         [0.1569, 0.1214, 0.0369,  ..., 0.7444, 0.2102, 0.6977],
         [0.0294, 0.9777, 0.6493,  ..., 0.0904, 0.3447, 0.

nn.Softmax
==========

The last linear layer of the neural network returns [logits]{.title-ref}
- raw values in \[-infty, infty\] - which are passed to the
[nn.Softmax](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html)
module. The logits are scaled to values \[0, 1\] representing the
model\'s predicted probabilities for each class. `dim` parameter
indicates the dimension along which the values must sum to 1.


In [12]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [13]:
print(softmax)
print(pred_probab)

Softmax(dim=1)
tensor([[0.0905, 0.1019, 0.1238, 0.1243, 0.0980, 0.0930, 0.0842, 0.1005, 0.1060,
         0.0779],
        [0.0871, 0.1119, 0.1147, 0.1326, 0.0871, 0.0968, 0.0969, 0.0844, 0.1048,
         0.0837],
        [0.0983, 0.1064, 0.1207, 0.1216, 0.0938, 0.0888, 0.0804, 0.0943, 0.1157,
         0.0800]], grad_fn=<SoftmaxBackward0>)


Model Parameters
================

Many layers inside a neural network are *parameterized*, i.e. have
associated weights and biases that are optimized during training.
Subclassing `nn.Module` automatically tracks all fields defined inside
your model object, and makes all parameters accessible using your
model\'s `parameters()` or `named_parameters()` methods.

In this example, we iterate over each parameter, and print its size and
a preview of its values.


In [14]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0100,  0.0339,  0.0198,  ...,  0.0094,  0.0073,  0.0184],
        [ 0.0225, -0.0259,  0.0269,  ...,  0.0030,  0.0220,  0.0095]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0222,  0.0220], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0030, -0.0102,  0.0292,  ...,  0.0075,  0.0110, -0.0320],
        [-0.0114,  0.0316, -0.0423,  ...,  0.0437, -0.0403,  0.0025]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | 

------------------------------------------------------------------------


Further Reading
===============

-   [torch.nn API](https://pytorch.org/docs/stable/nn.html)
